In [0]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import set_session, set_floatx
from skimage.transform import resize

In [0]:
cd ..

In [0]:
from tall_timbers_model.util.vis import view_seg_map

from tall_timbers_model.config import Config
from tall_timbers_model.models.floornet import SegModel

In [0]:
cap = cv2.VideoCapture("/path/01TP_extract.avi")
_, frame = cap.read()
out = cv2.VideoWriter("/hdd/videos/out_cvid.avi",
                      cv2.VideoWriter_fourcc("M", "J", "P", "G"), 10,
                      (frame.shape[1], frame.shape[0]))
xcount = 0

In [0]:

config = Config()
config.init_weights = None
config.image_size = (224, 224)
config.input_shape = (config.image_size[0], config.image_size[1], 3)

model = SegModel(config)
model.load_weights("/hdd/model.h5")

In [0]:
for i in range(200, 1000):
    ret, frame = cap.read()
    if not ret:
        break
    
    img = frame
    img = cv2.resize(img, (224, 224))
    
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x = cv2.Sobel(img2, cv2.CV_32F, 1, 0, ksize=3)
    x = cv2.convertScaleAbs(x) / 255.0
    x = x.reshape((x.shape[0], x.shape[1], 1))
    y = cv2.Sobel(img2, cv2.CV_32F, 0, 1, ksize=3)
    y = cv2.convertScaleAbs(y) / 255.0
    y = y.reshape((y.shape[0], y.shape[1], 1))
    z = cv2.Laplacian(img2, cv2.CV_32F)
    z = cv2.convertScaleAbs(z) / 255.0
    z = z.reshape((z.shape[0], z.shape[1], 1))
    image2 = np.concatenate((x, y, z), axis=2)
    image2 = cv2.resize(image2, (112, 112))
    nnInput = np.array(img, dtype=np.float32) / 255.0
    
    nnInput = 2 * (nnInput - 0.5)
    image2 = 2 * (image2 - 0.5)
    
    data = model.predict([nnInput[:, :, ::-1].reshape((1, 224, 224, 3)), image2.reshape((1, 112, 112, 3))])
    seg = data[0].argmax(axis=2).astype(np.float32)
    seg = resize(seg, (frame.shape[0], frame.shape[1])).reshape((frame.shape[0], frame.shape[1], 1))
    seg = np.around(seg)

    vis = view_seg_map(frame, seg, color=(0, 255, 0))
    
    print(vis.shape)
    out.write(vis)
    print(xcount)
    xcount = xcount + 1

cap.release()
out.release()

In [0]:
data = model.predict([nnInput.reshape((1, 224, 224, 3)), image2.reshape((1, 112, 112, 3))])

In [0]:
seg = data[0].argmax(axis=2)

In [0]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
vis = view_seg_map(img, seg, color=(0, 255, 0))